In [1]:
from os import listdir
import numpy as np
from collections import defaultdict
import json
import pandas as pd

filesList = listdir("TripAdvisorJson\\json")

businessIdMap = defaultdict(int)
businesses = defaultdict(list)

jsonpath = "TripAdvisorJson\\json\\"

allReviews=[]
fields = defaultdict(int)

for files,filenumber in zip(listdir(jsonpath),range(len(listdir(jsonpath)))):
    d=json.load(open(jsonpath+files,'r'))
    for review in d['Reviews']:
        userDict={}
        #print(review)
        #break;
        for key0 in review:
            if key0=='Ratings':
                for key1 in review[key0]:
                    userDict[key1]=review[key0][key1]
                    #print key1, val1
            else:
                userDict[key0]=review[key0]
                #print key0, val0
        userDict['HotelId'] = filenumber
        #print(userDict)
        allReviews+=[userDict]
        #break
    if(filenumber==5000):
        break  

In [2]:
print(len(allReviews))

546150


In [3]:
df=pd.DataFrame(allReviews)
del allReviews
print(df.columns)
try:
  df=df.drop(['Check in / front desk', 'Business service (e.g., internet access)','Business service', 'ReviewID', 'Sleep Quality'], axis=1)
except ValueError:
  df=df.drop(['Check in / front desk', 'ReviewID'], axis=1)
print(df.columns)
print(len(df.Author.unique()), len(df))
#df.ix[10000:10010]
df.head(5)

Index(['Author', 'AuthorLocation', 'Business service',
       'Business service (e.g., internet access)', 'Check in / front desk',
       'Cleanliness', 'Content', 'Date', 'HotelId', 'Location', 'Overall',
       'ReviewID', 'Rooms', 'Service', 'Sleep Quality', 'Title', 'Value'],
      dtype='object')
Index(['Author', 'AuthorLocation', 'Cleanliness', 'Content', 'Date', 'HotelId',
       'Location', 'Overall', 'Rooms', 'Service', 'Title', 'Value'],
      dtype='object')
412236 546150


,Author,AuthorLocation,Cleanliness,Content,Date,HotelId,Location,Overall,Rooms,Service,Title,Value
0,Nadine R,"Jacksonville, Florida",5,"Usually stay near the airport, but this trip w...","April 19, 2012",0,5,5.0,5,5,“Great place and location”,5
1,blt3116,"Chillicothe, Ohio",4,Stayed at this Hilton for 2 nights. It was lik...,"January 30, 2012",0,5,4.0,4,4,"“Nice stay, nice surroundings”",4
2,ExSpec5,Northern Virginia,5,"Stayed there one night, December 16, on the wa...","December 28, 2011",0,5,4.0,5,5,“Perfect for an overnight just off I-95”,3
3,kevinlynch3,Raleigh,5,I just stayed here last weekend and have alrea...,"December 23, 2011",0,5,5.0,5,5,“Great Hotel”,4
4,nobumponalog,"Belmont, MA",5,My mother who is 90 and I stayed one night on ...,"October 14, 2011",0,5,5.0,5,5,“good room for handicapped person”,4


In [4]:
listOfNans = df.isnull().sum(axis=1).tolist()
print(len(listOfNans) - np.count_nonzero(listOfNans))

365251


In [5]:
df = df.dropna(axis=0, how='any')
print(len(df))

365251


In [6]:
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer(r'\w+')
en_stop = stopwords.words('english')

from nltk.stem.porter import PorterStemmer

p_stemmer = PorterStemmer()
texts = []

C:\Users\Divya\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
for index, row in df.iterrows():
    i = row['Content']
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [x for x in tokens if not x in en_stop]
#     stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    texts.append(stopped_tokens)
print("Done")

Done


In [8]:
print(len(texts))
print(texts[0])
import csv

with open("TokenTexts.csv", "w", encoding='utf8') as f:
    writer = csv.writer(f)
    writer.writerows(texts)
    
print("Done")

365251
['usually', 'stay', 'near', 'airport', 'trip', 'business', 'north', 'side', 'balto', 'traffic', 'crazy', 'staying', 'elsewhere', 'hotel', 'perfect', 'hot', 'breakfast', 'wonderful', 'recommend', 'getting', 'breakfast', 'package', '2', 'room', 'hotel', 'area', 'restaurant', 'bar', 'area', 'whole', 'lot', 'good', 'restaurants', 'compared', 'staying', 'near', 'columbia', 'pikesville', '83', 'next', 'white', 'marsh', 'mall', 'huge', 'complex', 'shopping', 'including', 'ikea', 'beds', 'fabulous', 'adjust', 'side', 'firmness']
Done


In [9]:
del df

In [10]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(keep_n=20000)
dictionary.compactify()

corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary)

print("Done")

Done


In [13]:
ldamodel.save('ldamodel_5000')
ldamodel.print_topics(num_topics = 50, num_words = 20)

[(0,
  '0.015*"stay" + 0.013*"rooms" + 0.013*"smell" + 0.013*"even" + 0.011*"smoking" + 0.011*"rude" + 0.010*"would" + 0.010*"never" + 0.009*"non" + 0.009*"service" + 0.008*"worst" + 0.008*"one" + 0.008*"told" + 0.008*"asked" + 0.008*"manager" + 0.007*"bad" + 0.007*"smoke" + 0.006*"dirty" + 0.006*"terrible" + 0.006*"desk"'),
 (1,
  '0.086*"tram" + 0.086*"berlin" + 0.059*"u" + 0.030*"bahn" + 0.027*"cash" + 0.023*"bank" + 0.021*"shown" + 0.015*"centraal" + 0.014*"card" + 0.014*"aircon" + 0.014*"transport" + 0.013*"account" + 0.013*"basin" + 0.012*"platz" + 0.011*"exchange" + 0.011*"mould" + 0.009*"seedy" + 0.009*"dissapointed" + 0.009*"id" + 0.008*"pair"'),
 (2,
  '0.068*"booked" + 0.044*"reviews" + 0.034*"booking" + 0.020*"card" + 0.020*"website" + 0.015*"reception" + 0.014*"read" + 0.014*"book" + 0.013*"credit" + 0.013*"tripadvisor" + 0.012*"reading" + 0.012*"reservation" + 0.012*"site" + 0.012*"receptionist" + 0.011*"double" + 0.011*"com" + 0.011*"based" + 0.010*"online" + 0.009*"b" +

In [15]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis_data)

C:\Users\Divya\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [18]:
import pickle
with open('Visualization_5000_t_50_keepn_20k', 'wb') as output:
    pickle.dump(vis_data, output, pickle.HIGHEST_PROTOCOL)

We stayed here our zast night in Spain since it has a free airport shuttle. Our room was very clean and comfortable, with a nice bathroom and comfy beds. Front desk staff was ok during check in--not very welcoming, but not unfriendly. However, the cafetria and restaurant staff person was unhappy and rude for no apparent reason (we were friendly and low maintenance). As soon as we sat down he acted like our presence was an imposition--he was curt, forgot to bring parts of our order, and generally made us feel quite unwelcome. He served us on two different occasions and was the same during both. Also of note for travelers--the shuttle only has eight seats and goes once per hour, so get your reservation in early. We ended up getting our third time choice, meaning we got to the airport four hours before our departure. And, there is nothing near the hotel--if you don't have a car, you're stuck with the bad restaurant staff and mediocre food, or paying for a taxi that would need to be called. I was very disappointed--I expected more from a Crowne Plaza.


In [29]:
i = "We stayed here our zast night in Spain since it has a free airport shuttle. Our room was very clean and comfortable, with a nice bathroom and comfy beds. Front desk staff was ok during check in--not very welcoming, but not unfriendly. However, the cafetria and restaurant staff person was unhappy and rude for no apparent reason (we were friendly and low maintenance). As soon as we sat down he acted like our presence was an imposition--he was curt, forgot to bring parts of our order, and generally made us feel quite unwelcome. He served us on two different occasions and was the same during both. Also of note for travelers--the shuttle only has eight seats and goes once per hour, so get your reservation in early. We ended up getting our third time choice, meaning we got to the airport four hours before our departure. And, there is nothing near the hotel--if you don't have a car, you're stuck with the bad restaurant staff and mediocre food, or paying for a taxi that would need to be called. I was very disappointed--I expected more from a Crowne Plaza."
raw = i.lower()
tokens = tokenizer.tokenize(raw)
stopped_tokens = [x for x in tokens if not x in en_stop]
d = dict(ldamodel[dictionary.doc2bow(stopped_tokens)])
print([(k, d[k]) for k in sorted(d, key=d.get, reverse=True)])

[(16, 0.23453912228495202), (14, 0.15996467718683424), (29, 0.10824301014568929), (9, 0.09800147004574912), (34, 0.088666995906454382), (0, 0.0841810752767264), (46, 0.067952709939641101), (35, 0.060008616398603047), (40, 0.04093203795587283), (17, 0.024886508189697024), (26, 0.02414551580021574)]


In [30]:
vis_data = gensimvis.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis_data)

C:\Users\Divya\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [ ]:
import pickle
with open('Visualization_5000_t_50_keepn_20k', 'wb') as output:
    pickle.dump(vis_data, output, pickle.HIGHEST_PROTOCOL)